In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import tqdm
from requests_ip_rotator import ApiGateway

from fake_useragent import UserAgent
from itertools import cycle
import random 
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import csv


In [ ]:
url = "https://www.bayt.com/en/tunisia/jobs/?page=1"

service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service)

page = 1

df = []

while True:


    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    link = driver.find_elements(By.CSS_SELECTOR,".col.u-stretch.t-large.m0")
    
    for l in link:
        df.append(l.find_element(By.TAG_NAME,"a").get_attribute('href'))

   
    
    page += 1
    
    dff = pd.DataFrame(df, columns=['links'])
    
    dff.to_csv('bayt_jobs_links_today.csv', index = False)
    
    url = f"https://www.bayt.com/en/tunisia/jobs/?page={page}"
    
    if page == 50:
        break






In [5]:
urls = pd.read_csv('bayt_jobs_links_today.csv')

In [6]:

user = UserAgent()


storage = pd.DataFrame(columns=['post_date','job_title','job_description','company','location'])

def get_bayt(urls: pd.DataFrame) -> None:
    global storage
    for u in tqdm.tqdm(urls.links):
        headers =  {
            'User-Agent': user.random
        }

        url = u
        
       
        try:
            request = requests.get(url, timeout=3600 , headers= headers)
         
        except Exception as e :
            print(e)
            continue
        
        if request.status_code != 200:
      
 
            continue
        
            
        soup = BeautifulSoup(request.text, 'html.parser')
        time.sleep(2)
        
        post_date = soup.find('div', {'class': 't-small t-mute m10b'})
        
        job_title = soup.find('h1', {'class': 'h3 t-break'})

        job_description = soup.find('div', {'class': 't-break'})

        company = soup.find('a', {'class': 'is-black t-bold'})

        location = soup.find('span', {'class': 't-mute'})
        

   
        
        criteria = soup.find_all('dl', {'class': 'dlist is-spaced is-fitted t-small'})
        
        
        
       
       
       
        
        a = pd.DataFrame({'post_date':post_date.text.strip().split('\n')[0] if post_date else None,
                        'job_title':job_title.text.strip() if job_title else None, 
                        'job_description':job_description.text.strip() if job_description else None, 
                        'company':company.text.strip() if company else None,
                        'location':location.text.strip() if location else None, 
                        'industry': criteria[0].text.strip().split('\n')[5] if criteria else None,
    
                        
                        } , index=[0])

        

        storage = pd.concat([storage,a] ,axis=0)

        storage.to_csv('steven_bayt_jobs.csv', index = False)


    
   

In [7]:
get_bayt(urls)

 71%|███████   | 272/382 [13:47<04:16,  2.33s/it]

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|██████████| 382/382 [19:44<00:00,  3.10s/it]


In [16]:
import datetime
import re

def convert_relative_date(relative_date_str):
    """Converts a relative date string (e.g., "1 year ago", "2 months ago") to an actual date."""
    relative_date_str = relative_date_str.replace("+", "").strip()

    match = re.match(r"(\d+) (\w+) ago", relative_date_str)
    if not match:
        return None

    amount, unit = int(match.group(1)), match.group(2)

    now = datetime.datetime.now()

    if unit == "year" or unit == "years":
        delta = datetime.timedelta(days=amount * 365)
    elif unit == "month" or unit == "months":
        delta = datetime.timedelta(days=amount * 30)  # Approximation
    elif unit == "day" or unit == "days":
        delta = datetime.timedelta(days=amount)
    elif unit == "hour" or unit == "hours":
        delta = datetime.timedelta(hours=amount)
    elif unit == "minute" or unit == "minutes":
        delta = datetime.timedelta(minutes=amount)
    elif unit == "second" or unit == "seconds":
        delta = datetime.timedelta(seconds=amount)
    elif unit == "week" or unit == "weeks":
        delta = datetime.timedelta(weeks=amount)
    
    else:
        return None

    actual_date = now - delta
    return actual_date.strftime("%Y-%m")

convert_relative_date("7 weeks ago")

'2024-06'

In [17]:
bayt = pd.read_csv('bayt_jobs_27072024.csv')

In [18]:
bayt.head(3)

,post_date,job_title,job_description,company,location,industry
0,49 minutes ago,Goldstar / Inside Sales Representative - French,Job Title: Goldstar Inside Sales Representati...,National Pen,"Tunis, Tunisia",Other Business Support Services
1,37 minutes ago,Senior Recruiter,"Location: Tunis, TN, 1053 Senior RecruiterThe ...",Vistaprint,"Tunis, Tunisia",Other Business Support Services
2,20 minutes ago,Research Associate - Rangeland Landscape Ecology,Research Associate - Rangeland Landscape Ecolo...,CGIAR System Reference Group,Tunisia,Other Business Support Services


In [19]:
bayt.post_date = bayt.post_date.apply(lambda x: convert_relative_date(str(x)))

In [20]:
bayt.to_csv('bayt_jobs_27072024.csv',index=False)